In [ ]:
!pip install beautifulsoup4 requests pandas OpenCC

In [12]:
import pandas as pd
import requests as req
from bs4 import BeautifulSoup as bs
import opencc
from urllib.parse import quote, unquote
import re, os

# OpenCC
converter = opencc.OpenCC('s2tw.json')

# 建立存放資料的資料夾
path_folder = 'wikisource'
if not os.path.exists(path_folder):
    os.makedirs(path_folder)

# 網頁來源的前綴
prefix = 'https://zh.wikisource.org'

In [ ]:
df = pd.read_csv('./wikisource.csv')
df

In [ ]:
# 存放內文的 list
li_content = []

# 逐一取得內文
for idx, row in df.iterrows():
    # 請求網頁內容
    url = row['網址']
    res = req.get(url, timeout=10)

    # 解析網頁內容
    soup = bs(res.text, 'lxml')

    # 取得元素
    content_element = soup.select_one('div#mw-content-text')

    # 檢視內文
    # print( converter.convert(content.get_text()) )
    
    # 取得內文，並轉為繁體中文
    content = converter.convert(content_element.get_text())
    
    # 加入 list，之後整合到 DataFrame
    li_content.append(content)

    # 取得分類連結
    # alinks = soup.select('div#catlinks ul li a')
    # if len(alinks) > 0:
    #     for a in alinks:
    #         print(f'{a.get_text()}: {prefix}{unquote(a["href"])}')



# 將內文加入 DataFrame
df['內文'] = li_content

In [17]:
# 儲存成 Excel 檔
df.to_excel(f'{path_folder}/wikisource.xlsx', index=False)